Starting with the best result we have so far. We will take it as far as we can go in terms of accuracy, then start to study what makes each algorithm tick, as well as the predict probabilities aspect and if we want to go down that road. 

In [74]:
import numpy as np 
import pandas as pd
from helpers import * 
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
ufc = pd.read_csv('ufc-master.csv')

In [75]:
older_fighter = []

for i, row in ufc.iterrows():
    if row['R_age'] > row['B_age']:
        older_fighter.append(1)
    else:
        older_fighter.append(0)

ufc['Older_fighter'] = older_fighter

In [76]:
feats, targ = fine_tuning_setup(ufc, one_or_two=True)

In [77]:
feats['Older_fighter'] = ufc['Older_fighter']


In [72]:
for col in feats.columns:
    if col in ['R_age', 'B_age']:
        feats = feats.drop(col, axis=1)

In [81]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

bag = BaggingClassifier(n_estimators=150)
rf = RandomForestClassifier(n_estimators=175, max_features='log2', criterion='gini')
gb = GradientBoostingClassifier(learning_rate=0.2, criterion='squared_error', max_depth=15)
hgb = HistGradientBoostingClassifier(learning_rate=0.1, max_depth=150,max_leaf_nodes=None)
extra = ExtraTreesClassifier(n_estimators=75)
lgbm = LGBMClassifier(learning_rate=0.1, num_leaves=170)
xgb = XGBClassifier(reg_lambda=1.5, learning_rate=0.25)


ml_1 = {'bag' : bag,
        'extra' : extra, 
        'rf' : rf,
        'gb' : gb}

ml_2 = {'rf' : rf,
        'gb' : gb,
        'hgb' : hgb,
        'bag' : bag}

ml_3 = {'rf' : rf,
        'gb' : gb,
        'bag' : bag,
        'xgb' : xgb}

ml_4 = {'rf' : rf,
        'gb' : gb,
        'bag' : bag,
        'hgb' : hgb,
        'xgb' : xgb}

ml_all = {'rf' : rf,
        'gb' : gb,
        'bag' : bag,
        'hgb' : hgb,
        'xgb' : xgb,
        'lgbm' : lgbm,
        'extra' : extra}


In [80]:
simulate(feats, targ, ml_1)

bag
0.7250755287009063
extra
0.7175226586102719
rf
0.7235649546827795
gb
0.7099697885196374
hgb
0.6978851963746223


In [82]:
acc = execute_voting_clf(feats, targ, ml_1)

In [83]:
acc

0.7129909365558912

In [37]:
acc = custom_ensemble_execute(feats, targ, ml_3, tipping_point=2)

In [38]:
acc

0.7265861027190332